# Lesson 1 - BOLDvessel

The `BOLDvessel` module contains the objects which define perturbers in continuous-space simulations. To begin we first import the module as follows (and NumPy for array creation):

In [ ]:
from BOLDswimsuite import BOLDvessel
import numpy as np

We must now choose what type of vessel we need for our purposes, which includes both the shape and number of dimensions:

3D:
- Infinite cylinder: `BOLDvessel.InfiniteCylinder3D`
- Sphere: `BOLDvessel.Sphere3D`

2D:
- Infinite Cylinder: `BOLDvessel.InfiniteCylinder2D`
- In-plane sphere: `BOLDvessel.Sphere2D`

All of these objects are created and function similarily, so we will use `BOLDvessel.InfiniteCylinder3D` to explain how to create and use these objects.

## Creating a vessel object

There are two ways to create a vessel object:
1. Manually define all the parameters
2. Randomly generate the perturber according to some pararmeters

We will begin with the first. The default constructor for `BOLDvessel.InfiniteCylinder3D` requires the following arguments:

- diameter : float, vessel diameter (mm)
- theta : float, zenith angle of the vessel direction (radians)
- phi : float, azimuth angle of the vessel direction (radians)
- origin : np.ndarray, cartesian coordinates of the vessel origin (mm)
- dchi : float, susceptibility difference between the vessel and the surrounding tissue (cgs units)
- permeation_probability : float, probability for a spin to permeate through the vesel wall (fraction of 1). Default is 0.
- label : str, string to identify the vessel. Default is ''.

>Note: It is good practice whenever making float arrays to add a decimal point (i.e. "1." instead of "1"), so that the array is not accidentally interpreted as an integer, which results in a runtime error.

In [ ]:
# creating some vessel with arbitrary parameters

vessel = BOLDvessel.InfiniteCylinder3D(
    diameter=0.1, #mm
    theta=np.pi/3, #radians
    phi=np.pi/4,
    origin=np.array([0., 0., 0.]), #mm
    dchi=3e-8, #cgs units
    permeation_probability=0, #probability
    label='vein'
)

> Note: the `label` parameter is mainly useful when categorizing groups of vessels which would share the same label. It is also an optional parameter.

Now we look at the second way to create vessels, using the class method `from_random`. This is used to randomly position vessels within a bounding box or voxel. It has the following arguments:

- diameter : float, vessel diameter (mm) 
- dchi : float, susceptibility difference between the vessel and the surrounding tissue (cgs units)
- voxel_size : float, size of the bounding box (mm)
- permeation_probability : float, probability for a spin to permeate through the vesel wall (fraction of 1). Default is 0.
- label : str, string to identify the vessel. Default is ''.
- rng : np.random.Generator, random generator to be used for random number generation. Default is np.random.default_rng(). 

>Note: The `rng` argument is ignored in the example. It is used for seeding the random numbers.

In [ ]:
# Running multiple times will provide more random vessels.

random_vessel = BOLDvessel.InfiniteCylinder3D.from_random(
    diameter=0.1, #mm
    dchi=3e-8, #cgs
    voxel_size=10, #mm
    permeation_probability=0,
    label='vein'
)

print(f"Origin: {random_vessel.origin}")
print(f"Theta: {random_vessel.theta}")
print(f"Phi: {random_vessel.phi}")


>Note: All constructors have a docstring which can be accessed with the `help` function. Information on the created object and a description of each required argument can be found in the docstring.

In [ ]:
#the alternate constructor for InfiniteCylinder3D
help(BOLDvessel.InfiniteCylinder3D.from_random)

## Vessel Methods

Now that we can create vessels, there are several methods we can use to obtain information from them.

First there is the `is_IV` method, which, given an array of positions (in mm), will return a boolean array indicating whether a position is EV or IV.
For example, we know that the position `[0.01,0.01,0.01]` is intravascular to the vessel we created, and the position `[1000,1000,1000]` is extravascular.

> Note: all arguments named `positions` require a (N, d) array of values, where N is the number of positions and d is the number of dimensions. In this example we are using a 3D vessel and we have 2 positions, so the array will be of shape (2, 3).

In [ ]:
positions = np.array(
    [[0.01, 0.01, 0.01], #first position at [0.01,0.01,0.01] (mm)
     [1000, 1000, 1000]] #second position at [1000,1000,1000] (mm)
)

vessel.is_IV(positions)

The method returned just as expected, `True` or intravascular for the first position, and `False` or extravascular for the second one.

Next there are the `dBz_EV` and `dBz_IV` methods which provide the extravascular and intravascular magnetic field offsets, respectively. The arguments for `dBz_EV` are the positions where the dBz will be calculated and the B0 magnetic field strength (in Tesla). For `dBz_IV`, only B0 is required.

In [ ]:
B0 = 3

vessel.dBz_EV(positions, B0)

Here we get the extravascular dBz for all of the positions given.

In [ ]:
vessel.dBz_IV(B0)

Here we get the intravascular dBz (only one value is returned as it is position-independent).

There is also the convenience function `is_IV_dBz` which provides all the information above with only one method call. This method requires the same arguments as `dBz_EV`, and returns a Tuple with the output of `is_IV`, `dBz_EV` and `dBz_IV`. When all three are needed, this method is more computationally efficient and so should be used.

In [ ]:
is_IV, dBz_EV, dBz_IV = vessel.is_IV_dBz(positions, B0)
print(f'is_IV: {is_IV}')
print(f'dBz_EV: {dBz_EV}')
print(f'dBz_IV: {dBz_IV}')

>Note: All methods also have a docstring which can be accessed with the `help` function. Information on the method purpose and a description of each required argument can be found in the docstring.

In [ ]:
#the is_IV_dBz method of InfiniteCylinder3D
help(BOLDvessel.InfiniteCylinder3D.is_IV_dBz)

There are also some other utilities that are useful for other things.
`volume_fraction` takes in the size of a voxel, and outputs an estimated volume fraction of the voxel occupied by the vessel (voxel is centered arround zero).

In [ ]:
voxel_size = 10 #mm
vessel.volume_fraction(voxel_size)

We can also check whether two vessels intersect with `intersects` (only works when both are the same type of vessel).

In [ ]:
# creating another vessel
vessel2 = BOLDvessel.InfiniteCylinder3D(
    diameter=2, #mm
    theta=-np.pi/3, #radians
    phi=np.pi/2,
    origin=np.array([1., 1., 0.]), #mm
    dchi=3e-8, #cgs units
    permeation_probability=0, #probability
    label='artery'
)

vessel.intersects(vessel2)